In [1]:
%load_ext autoreload
%autoreload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import prim
import seaborn as sns
#from sklearn import mixture
import os
#from clustering_and_prim_tools import *
#from SALib.analyze import sobol
#from SALib.sample import saltelli

In [3]:
%matplotlib inline

In [4]:
sns.set_context("poster",rc={"font.size": 28})
sns.set_style("whitegrid")

In [5]:
rootfolder = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\transport_cired_output_data\\"

invfolder = rootfolder+"outputs_investments_module_billionusd/"
imaclimfolder = rootfolder+"outputs_Imaclim/"
physicalinfrafolder = rootfolder+"outputs_investments_module_physical_stock/"


In [6]:
resilience_folder = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\resilience\\Elco latest results\\"
resilience_folder2 = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\resilience\\"

In [7]:
country_classification = pd.read_excel('C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\country_classification_2019.xls','List of economies')

In [8]:
os.listdir(resilience_folder2)

['bridges_wbregion.csv',
 'CBA_AF.csv',
 'CBA_CF.csv',
 'CBA_FU.csv',
 'CBA_PU.csv',
 'cost_diff.xlsx',
 'cost_diff_wash.xlsx',
 'Elco latest results',
 'exposed_1_100.csv',
 'exposed_at_least_one.csv',
 'power robcosts_Julie.xlsx',
 'power robcosts_Julie2.xlsx',
 'power_expo_risk_generation.xlsx',
 'rail_wbregion.csv',
 'rail_wbregion_exposure_risk.csv',
 'road spending GDP.xlsx',
 'road spending.xlsx',
 'roads_wbregion.csv',
 'roads_wbregion_exposure_risk.csv',
 'roads_wbregion_test.xlsx',
 'UrbanSanitation.xlsx']

In [9]:
CBA_CF = pd.read_csv(resilience_folder2+'CBA_CF.csv').rename(columns={"length_exp":"length_exp_CF"})
CBA_FU = pd.read_csv(resilience_folder2+'CBA_FU.csv').rename(columns={"length_exp":"length_exp_FU"})
CBA_PU = pd.read_csv(resilience_folder2+'CBA_PU.csv').rename(columns={"length_exp":"length_exp_PU"})

CBA_AF = pd.read_csv(resilience_folder2+'CBA_AF.csv')

In [10]:
CBA_AF.sample(3)

,Unnamed: 0,infra_type,perc_50,rel_risk,all_risk_road,infra_value,wbincome_x,main,length,length_exp,ISO_3digit,wbincome_y,wbincome,upgrade,npv_loss,bca,nb
46828,JPN_44_21_1,secondary,515.698737,0.002664,22.075720,34767.595084,HIC,5789.279787,22.075720,8.388174,JPN,HIO,HIC,695.351902,6341.069383,0.977861,-143.562306
83258,SLV_6_13_1,secondary,602.623746,0.000153,11.587393,1406.341679,LMC,171.927552,11.587393,0.116469,SLV,LMI,LMC,28.126834,5570.796718,10.000000,5370.742332
47017,JPN_46_20_1,primary,3547.604387,0.002416,81.499610,433110.876758,HIC,95350.950524,81.499610,21.457084,JPN,HIO,HIC,8662.217535,54362.776939,0.522653,-49650.391120


In [11]:
CBA_AF = CBA_AF.merge(country_classification.rename(columns={'Code':'ISO_3digit'})[['ISO_3digit','Region']],on='ISO_3digit',how='left')

In [12]:
exposed_all_floods = CBA_AF.set_index(["ISO_3digit",'Region','wbincome'])[["length","length_exp"]].sum(level=["ISO_3digit",'Region','wbincome'])

In [13]:
exposed_all_floods['share_exposed'] = exposed_all_floods.length_exp/exposed_all_floods.length

In [14]:
exposed_all_floods_wbregion = exposed_all_floods[["length","length_exp"]].sum(level="Region")
exposed_all_floods_wbregion['share_exposed'] = exposed_all_floods_wbregion.length_exp/exposed_all_floods_wbregion.length

In [15]:
exposed_all_floods_wbincome = exposed_all_floods[["length","length_exp"]].sum(level="wbincome")
exposed_all_floods_wbincome['share_exposed'] = exposed_all_floods_wbincome.length_exp/exposed_all_floods_wbincome.length

In [16]:
dict_region = {'Latin America & Caribbean':'LAM', 'Europe & Central Asia':'CIS',
       'Middle East & North Africa':"MAF", 'East Asia & Pacific':"ASIA",
       'North America':"OCDE", 'South Asia':'ASIA', 'Sub-Saharan Africa':'MAF'}

In [17]:
exposed_all_floods;

In [18]:
exposed_all_floods = exposed_all_floods.reset_index("Region")
exposed_all_floods['imaclim_region'] = exposed_all_floods.Region.replace(dict_region)

exposed_all_floods_imaclim = exposed_all_floods.set_index(['Region','imaclim_region'],append=True)[["length","length_exp"]].sum(level="imaclim_region")
exposed_all_floods_imaclim['share_exposed'] = exposed_all_floods_imaclim.length_exp/exposed_all_floods_imaclim.length

In [19]:
exposed_all_floods_imaclim

,length,length_exp,share_exposed
imaclim_region,,,
LAM,5.720895e+06,4.022824e+05,0.070318
ASIA,9.998770e+06,1.298429e+06,0.129859
MAF,6.342259e+06,3.458822e+05,0.054536
CIS,1.145742e+07,6.896106e+05,0.060189
OCDE,1.168767e+07,6.560104e+05,0.056128


In [20]:
exposed_floods = CBA_CF[["Unnamed: 0","ISO_3digit","length","length_exp_CF"]].merge(CBA_FU[["Unnamed: 0","length_exp_FU"]],on=["Unnamed: 0"]).merge(CBA_PU[["Unnamed: 0","length_exp_PU"]],on=["Unnamed: 0"])

In [21]:
exposed_floods = exposed_floods.merge(country_classification.rename(columns={'Code':'ISO_3digit'})[['ISO_3digit','Region']],on='ISO_3digit',how='left')

In [22]:
exposed_floods['imaclim_region'] = exposed_floods.Region.replace(dict_region)

In [23]:
exposed_floods_imaclim = exposed_floods.set_index('imaclim_region')[['length','length_exp_CF',"length_exp_FU","length_exp_PU"]].sum(level='imaclim_region')
exposed_floods_wbreg = exposed_floods.set_index('Region')[['length','length_exp_CF',"length_exp_FU","length_exp_PU"]].sum(level='Region')

In [24]:
exposed_floods_imaclim['share_exposed'] = exposed_floods_imaclim[['length_exp_CF',"length_exp_FU","length_exp_PU"]].sum(axis=1)/exposed_floods_imaclim['length']
exposed_floods_wbreg['share_exposed'] = exposed_floods_wbreg[['length_exp_CF',"length_exp_FU","length_exp_PU"]].sum(axis=1)/exposed_floods_wbreg['length']

In [25]:
exposed_floods_wbreg

,length,length_exp_CF,length_exp_FU,length_exp_PU,share_exposed
Region,,,,,
Sub-Saharan Africa,1.599550e+06,5089.583262,2.304150e+04,3.399187e+04,0.038838
Europe & Central Asia,1.809133e+07,929980.789599,5.950536e+05,4.440217e+05,0.108840
Middle East & North Africa,2.270970e+06,53511.178031,7.973716e+04,3.583158e+04,0.074453
Latin America & Caribbean,3.890118e+06,29968.894448,1.390783e+05,1.531675e+05,0.082829
East Asia & Pacific,1.796782e+07,839069.096909,1.123067e+06,2.053144e+06,0.223471
South Asia,1.959134e+06,21469.245345,5.881345e+04,9.839042e+04,0.091200
North America,1.132532e+07,267381.220175,4.733918e+05,3.773661e+05,0.098729


In [26]:
exposed_floods_country = exposed_floods.set_index('ISO_3digit')[['length','length_exp_CF',"length_exp_FU","length_exp_PU"]].sum(level='ISO_3digit')

In [27]:
exposed_floods_country['share_exposed'] = exposed_floods_country[['length_exp_CF',"length_exp_FU","length_exp_PU"]].sum(axis=1)/exposed_floods_country['length']

In [28]:
exposed_floods_country.to_csv("country_roads_flood_exposure.csv")

In [29]:
exposed_floods_country.share_exposed.describe()

count    127.000000
mean       0.098213
std        0.099525
min        0.006780
25%        0.047264
50%        0.072143
75%        0.120721
max        0.922271
Name: share_exposed, dtype: float64

In [30]:
exposed_floods_country[exposed_floods_country.share_exposed>0.2]

,length,length_exp_CF,length_exp_FU,length_exp_PU,share_exposed
ISO_3digit,,,,,
ALB,6.808661e+04,4825.130464,6419.250497,3429.137410,0.215513
BEL,3.964480e+05,46395.240104,25235.838704,8556.004229,0.202264
BGD,2.302448e+05,11479.487498,14051.354867,21659.621086,0.204958
CHN,4.533437e+06,299088.031349,487527.308461,502261.048719,0.284304
DEU,9.264534e+05,168365.782235,68326.350749,14642.009942,0.271286
IRQ,6.850095e+03,330.506729,1367.841145,57.799100,0.256368
JPN,4.780649e+06,370644.086992,283406.111947,817640.852044,0.307843
NLD,5.878246e+05,381511.858878,143003.707000,17618.037067,0.922271
TWN,9.332382e+05,38295.322010,34956.728777,121712.580700,0.208912


In [31]:
exposed_floods_country.loc["MDG"]

length           67279.833640
length_exp_CF      167.390647
length_exp_FU     3431.534692
length_exp_PU     2673.120493
share_exposed        0.093223
Name: MDG, dtype: float64

In [32]:
exposed_floods_imaclim

,length,length_exp_CF,length_exp_FU,length_exp_PU,share_exposed
imaclim_region,,,,,
MAF,3.870520e+06,58600.761293,1.027787e+05,6.982345e+04,0.059734
CIS,1.809133e+07,929980.789599,5.950536e+05,4.440217e+05,0.108840
LAM,3.890118e+06,29968.894448,1.390783e+05,1.531675e+05,0.082829
ASIA,1.992696e+07,860538.342253,1.181880e+06,2.151534e+06,0.210466
OCDE,1.132532e+07,267381.220175,4.733918e+05,3.773661e+05,0.098729


In [33]:
exposed_floods_imaclim[['length_exp_CF',"length_exp_FU","length_exp_PU"]].div(exposed_floods_imaclim['length'],axis=0)

,length_exp_CF,length_exp_FU,length_exp_PU
imaclim_region,,,
MAF,0.015140,0.026554,0.018040
CIS,0.051405,0.032892,0.024543
LAM,0.007704,0.035752,0.039373
ASIA,0.043185,0.059311,0.107971
OCDE,0.023609,0.041799,0.033321


In [34]:
exposed = pd.read_csv(resilience_folder2+'exposed_1_100.csv')

In [35]:
exposed.sample(5)

,Unnamed: 0,road_type,region,country,continent,length,length_exp,wbincome
89283,89283,other,CHN_23_7_1,CHN,Asia,17.943428,0.419252,UMC
69899,69899,primary,SYR_8_4_1,SYR,Asia,119.066161,12.379904,LIC
120254,120254,primary,BRA_25_635_1,BRA,South-America,36.546219,2.468303,UMC
118143,118143,tertiary,ROU_5_83_1,ROU,Europe,29.972273,0.877288,UMC
99835,99835,secondary,THA_32_13_1,THA,Asia,58.419990,5.129867,UMC


In [36]:
exposed = exposed.merge(country_classification.rename(columns={'Code':'country'})[['country','Region']],on='country',how='left')

In [37]:
exposed.Region.unique()

array(['Latin America & Caribbean', 'Europe & Central Asia',
       'Middle East & North Africa', 'East Asia & Pacific',
       'North America', 'South Asia', 'Sub-Saharan Africa', nan],
      dtype=object)

In [38]:
exposed['imaclim_region'] = exposed.Region.replace(dict_region)

In [39]:
exposed_imaclim = exposed.set_index('imaclim_region')[['length','length_exp']].sum(level='imaclim_region')

In [40]:
exposed_imaclim['share_exposed'] = exposed_imaclim['length_exp']/exposed_imaclim['length']

In [41]:
exposed_imaclim

,length,length_exp,share_exposed
imaclim_region,,,
LAM,6.264128e+06,278533.624083,0.044465
CIS,1.914540e+07,671840.619202,0.035091
MAF,7.918546e+06,345555.908492,0.043639
ASIA,1.139793e+07,752360.143821,0.066008
OCDE,1.406668e+07,498106.859170,0.035410


In [42]:
wb_group_risk = pd.read_csv(resilience_folder+"wb_group_risk.csv").rename(columns={'Unnamed: 0':'hazard','Unnamed: 1':"asset_type"})
wb_region_prot_base = pd.read_csv(resilience_folder+"wb_region_prot_base.csv").rename(columns={'Unnamed: 0':'hazard'})
wb_region_prot_improved = pd.read_csv(resilience_folder+"wb_region_prot_improved.csv").rename(columns={'Unnamed: 0':'hazard'})

In [43]:
wb_group_risk.sample(3)

,hazard,asset_type,GroupName,perc_0,perc_20,perc_40,perc_50,perc_60,perc_80,perc_100
96,EQ,Primary Roads,SAS,1.245062e+06,3.504258e+06,5.400943e+06,6.276311e+06,7.927029e+06,1.489537e+07,2.437547e+07
169,PU,Bridge_Road,SAS,1.374531e+06,2.870843e+06,4.928311e+06,7.347280e+06,1.028094e+07,2.753607e+07,7.141889e+07
164,PU,Bridge_Rail,ECA,0.000000e+00,2.937506e+02,4.971163e+02,5.670558e+02,1.481115e+04,4.310929e+04,9.984991e+04


In [44]:
wb_group_risk.set_index(['hazard','asset_type','GroupName']).sum()*10**(-9)

perc_0       4.450087
perc_20      6.798967
perc_40     10.334737
perc_50     12.547806
perc_60     15.316866
perc_80     22.332173
perc_100    36.855102
dtype: float64

In [45]:
wb_group_risk.hazard.unique()

array(['CF', 'Cyc', 'EQ', 'FU', 'PU'], dtype=object)

In [46]:
wb_group_risk.set_index(['hazard','asset_type','GroupName']).drop(['EQ'],level='hazard').sum()*10**(-9)

perc_0       4.048603
perc_20      6.133197
perc_40      9.284744
perc_50     11.310752
perc_60     13.898237
perc_80     20.170596
perc_100    32.598817
dtype: float64

In [47]:
wb_region_prot_base.set_index(['hazard','GroupName']).sum()*10**(-9)

perc_0       3.763746
perc_20      5.496053
perc_40      8.255442
perc_50      9.951583
perc_60     12.241325
perc_80     17.342847
perc_100    27.351200
dtype: float64

In [48]:
wb_region_prot_improved.set_index(['hazard','GroupName']).sum()*10**(-9)

perc_0       1.983244
perc_20      2.977813
perc_40      4.677276
perc_50      5.782950
perc_60      7.135032
perc_80     10.058273
perc_100    15.809623
dtype: float64

# data prep (skip and go to next comment if this has run once already)

In [6]:
os.listdir(imaclimfolder)

['results_transports_Imaclim_ASIA.csv',
 'results_transports_Imaclim_CIS.csv',
 'results_transports_Imaclim_LAM.csv',
 'results_transports_Imaclim_MAF.csv',
 'results_transports_Imaclim_OCDE.csv',
 'results_transports_Imaclim_WORLD.csv']

In [7]:
pd.options.display.max_seq_items = 2000

In [8]:
all_ima_data = pd.DataFrame()
for myfilename in os.listdir(imaclimfolder):
    if "WORLD" in myfilename:
        continue
    myfile = pd.read_csv(imaclimfolder+"\\"+myfilename,sep=";")
    myinfo = myfilename.replace(".csv",'').split("_")
    myfile["region"] = myinfo[3]
    all_ima_data = all_ima_data.append(myfile)

In [9]:
all_ima_data.Variable.unique()

array(['Population', 'CO2_em_tot', 'CO2_em_trans', 'PIB',
       'passenger_activity', 'freight_activity', 'pkm_OT', 'pkm_auto',
       'pkm_air', 'fret_air', 'fret_OT', 'fret_mer'], dtype=object)

In [10]:
indexes_ima = ['//combi', 'ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'ind_taxexo', 'ind_recycl',
       'ind_taxmax', 'Variable', 'Unit','region']

In [11]:
indexes_ima2 = ['ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'ind_taxexo', 'ind_recycl',
       'ind_taxmax', 'Variable','region','year']

In [12]:
all_ima_data = all_ima_data.set_index(indexes_ima)
all_ima_data.columns.name='year'
all_ima_data = all_ima_data.stack('year')
all_ima_data.name = 'value'
all_ima_data = all_ima_data.reset_index('year')
all_ima_data['year'] = all_ima_data.year.astype("int64")
all_ima_data = all_ima_data.loc[(all_ima_data.year>2014)&(all_ima_data.year<2031),:]

In [13]:
all_ima_data.set_index("year",append=True).unstack("Variable")["value"]["PIB"].dropna();

In [14]:
out = all_ima_data.reset_index().drop(['Unit','//combi'],axis=1).set_index(indexes_ima2).unstack('Variable').value

In [15]:
out.columns

Index(['CO2_em_tot', 'CO2_em_trans', 'PIB', 'Population', 'freight_activity',
       'fret_OT', 'fret_air', 'fret_mer', 'passenger_activity', 'pkm_OT',
       'pkm_air', 'pkm_auto'],
      dtype='object', name='Variable')

In [16]:
to_merge = out.reset_index().set_index([x for x in indexes_ima2 if x not in ["Variable"]])["PIB"]

In [17]:
indexes_inv = ['combi', 'ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'modal_scenario',
        'ur_ideal', 'urt_ideal','delay','costs_evolution_road', 'costs_evolution_rail']

In [18]:
os.listdir(invfolder)

['ASIA', 'CIS', 'LAM', 'MAF', 'OCDE']

In [20]:
os.listdir(invfolder+'ASIA')

['cost_maint_air_ASIA.csv',
 'cost_maint_brt_ASIA.csv',
 'cost_maint_hsr_ASIA.csv',
 'cost_maint_rail_ASIA.csv',
 'cost_maint_road_ASIA.csv',
 'cost_new_air_ASIA.csv',
 'cost_new_brt_ASIA.csv',
 'cost_new_hsr_ASIA.csv',
 'cost_new_rail_ASIA.csv',
 'cost_new_road_ASIA.csv']

In [26]:
all_data_usd = pd.DataFrame()
all_data_pct = pd.DataFrame()

for reg in os.listdir(invfolder):
    for myfilename in os.listdir(invfolder+reg):
        myfile = pd.read_csv(invfolder+reg+"\\"+myfilename,sep=";")
        myfile = myfile.set_index(indexes_inv)
        myfile.columns.name='year'
        myfile = myfile.stack()
        myfile.name = 'value'
        myfile = myfile.reset_index()
        myfile["year"] = myfile.year.astype("int64")
        myfile = myfile.loc[myfile.year<2031,:]
                
        myinfo = myfilename.replace(".csv",'').split("_")
        myfile["costtype"] = myinfo[1]
        myfile["transpmode"] = myinfo[2]
        myfile["region"] = reg
        
        myfile = myfile.merge(to_merge.reset_index(),on=['ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F','region', 'year'],how="left")
        
        global_file = myfile.set_index(indexes_inv+["costtype","transpmode","region","year"])[["PIB","value"]]
        
        global_file["divided_by_gdp"] = global_file.value/global_file.PIB

        global_file2 = global_file.reset_index(["ur_ideal","year"])

        global_file2["discount_factor"] = (1+0.06)**(1-(global_file2.year.astype(int)-2014))
        global_file2['disc_value'] =  global_file2['value']*global_file2["discount_factor"]

        global_file2 = global_file2.loc[global_file2.ur_ideal>300,:]
        global_file2 = global_file2.set_index(["ur_ideal","year"],append=True)
        
        
        all_data_usd = all_data_global.append(global_file2.disc_value.sum(level=indexes_inv+["region","costtype","transpmode"]).to_frame())
        all_data_pct = all_data_global.append(global_file2.divided_by_gdp.mean(level=indexes_inv+["region","costtype","transpmode"]).to_frame())

In [27]:
all_data_global2 = all_data_global.sum(level = indexes_inv+["costtype","transpmode","year"])

In [46]:
all_data_global.to_csv("reg_data_for_resilient_report.csv")

In [36]:
all_data_global2.to_csv("for_resilient_report.csv")

In [11]:
all_data_reg = pd.read_csv("reg_data_for_resilient_report.csv").set_index(indexes_inv+["region","costtype","transpmode","year"])

In [12]:
all_data_reg["divided_by_gdp"] = all_data_reg.value/all_data_reg.PIB

all_data_reg2 = all_data_reg.reset_index(["ur_ideal","year"])

all_data_reg2["discount_factor"] = (1+0.06)**(1-(all_data_reg2.year.astype(int)-2014))
all_data_reg2['disc_value'] =  all_data_reg2['value']*all_data_reg2["discount_factor"]


all_data_reg2 = all_data_reg2.loc[all_data_reg2.ur_ideal>300,:]
all_data_reg2 = all_data_reg2.set_index(["ur_ideal","year"],append=True)

In [13]:
reg_usd_data = all_data_reg2.disc_value.sum(level=indexes_inv+["region","costtype","transpmode"])
reg_pct_data = all_data_reg2.divided_by_gdp.mean(level=indexes_inv+["region","costtype","transpmode"])

In [14]:
reg_usd_data.to_csv("reg_usd_data.csv")
reg_pct_data.to_csv("reg_pct_data.csv")

# Start here if the file exists already

In [49]:
indexes_inv = ['combi','ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'modal_scenario',
        'ur_ideal', 'urt_ideal','delay','costs_evolution_road', 'costs_evolution_rail']

In [50]:
reg_usd_data = pd.read_csv("reg_usd_data.csv", names = indexes_inv+["region","costtype","transpmode","disc_value"]).set_index(indexes_inv+["region","costtype","transpmode"])
reg_pct_data = pd.read_csv("reg_pct_data.csv", names = indexes_inv+["region","costtype","transpmode","divided_by_gdp"]).set_index(indexes_inv+["region","costtype","transpmode"])

In [51]:
reg_pct_data.sample(5)

,,,,,,,,,,,,,,,,,divided_by_gdp
combi,ind_climat,ind_growth,ind_mitig,ind_trans_A,ind_trans_S,ind_trans_I,ind_trans_F,modal_scenario,ur_ideal,urt_ideal,delay,costs_evolution_road,costs_evolution_rail,region,costtype,transpmode,
84,0,3,2,1,1,2,2,1,900,30000,35,2,1,MAF,new,brt,0.000798
109,2,1,1,2,2,1,1,1,900,5000,35,1,1,OCDE,maint,road,0.008954
184,2,3,2,1,2,2,2,0,900,30000,65,1,2,ASIA,new,road,0.000000
125,2,1,2,2,2,1,1,0,600,5000,35,0,2,MAF,maint,rail,0.000537
377,4,3,2,2,1,1,1,1,600,5000,35,0,2,LAM,new,brt,0.000354


In [52]:
reg_pct_data.reset_index("transpmode").transpmode.unique()

array(['air', 'brt', 'hsr', 'rail', 'road'], dtype=object)

## Adding the resilience extra cost

In [64]:
cost_diff = pd.read_excel(resilience_folder2+"cost_diff.xlsx").fillna(0)

In [131]:
cost_diff = cost_diff[cost_diff.hazard=='flood']

In [132]:
exposed_all_floods_imaclim.share_exposed.reset_index().rename(columns={'imaclim_region':'region'})

,region,share_exposed
0,LAM,0.070318
1,ASIA,0.129859
2,MAF,0.054536
3,CIS,0.060189
4,OCDE,0.056128


In [133]:
reg_pct_data_res = reg_pct_data.reset_index().copy()

reg_pct_data_res = reg_pct_data_res.merge(cost_diff, on=["transpmode","costtype"], how="left").merge(exposed_all_floods_imaclim.share_exposed.reset_index().rename(columns={'imaclim_region':'region'}),on='region')

reg_pct_data_res["divided_by_gdp_res"] = reg_pct_data_res["divided_by_gdp"]*(1+reg_pct_data_res["cost_increase"]*reg_pct_data_res["share_exposed"])
reg_pct_data_res["divided_by_gdp_res_all"] = reg_pct_data_res["divided_by_gdp"]*(1+reg_pct_data_res["cost_increase"])

reg_pct_data_res["initial_risk"] = reg_pct_data_res["divided_by_gdp"]*reg_pct_data_res["initial_risk"]*reg_pct_data_res["share_exposed"]
reg_pct_data_res["final_risk"] = reg_pct_data_res["divided_by_gdp"]*reg_pct_data_res["final_risk"]*reg_pct_data_res["share_exposed"]

In [134]:
reg_pct_data_res['cost_diff'] = reg_pct_data_res["divided_by_gdp_res"]-reg_pct_data_res["divided_by_gdp"]
reg_pct_data_res['cost_diff_all'] = reg_pct_data_res["divided_by_gdp_res_all"]-reg_pct_data_res["divided_by_gdp"]

In [135]:
reg_pct_data_res = reg_pct_data_res.set_index(indexes_inv+["region","costtype","transpmode"])

In [136]:
100*reg_pct_data_res.sum(level=indexes_inv+["region","costtype"]).cost_diff.unstack("costtype")['new'].describe()

count    2.073600e+07
mean     2.235538e-02
std      2.434164e-02
min      8.759089e-09
25%      3.070986e-03
50%      1.001503e-02
75%      4.361456e-02
max      1.300177e-01
Name: new, dtype: float64

In [137]:
100*reg_pct_data_res.sum(level=indexes_inv+["region","costtype"]).cost_diff_all.unstack("costtype")['new'].describe()

count    2.073600e+07
mean     2.881325e-01
std      3.081640e-01
min      1.455265e-07
25%      4.492746e-02
50%      1.620383e-01
75%      4.660491e-01
max      1.307632e+00
Name: new, dtype: float64

In [138]:
reg_pct_alltype_all_modes = reg_pct_data_res.sum(level=indexes_inv+["region"])

In [139]:
reg_pct_data_res_all_modes = reg_pct_data_res.sum(level=indexes_inv+["region","costtype"])
reg_pct_data_res_all_modes = reg_pct_data_res_all_modes.reset_index('costtype')[reg_pct_data_res_all_modes.reset_index('costtype').costtype=='new']

## % increase in capital investment needs due to increased resilience everywhere

In [140]:
100*(reg_pct_data_res_all_modes['cost_diff_all']/reg_pct_data_res_all_modes['divided_by_gdp']).describe()

count    2.073600e+07
mean     2.414922e+01
std      1.815430e+01
min      3.836243e-04
25%      5.157421e+00
50%      2.323097e+01
75%      4.382271e+01
max      4.934322e+01
dtype: float64

## % increase in capital investment needs due to increased resilience only in exposed areas

In [141]:
100*(reg_pct_data_res_all_modes['cost_diff']/reg_pct_data_res_all_modes['divided_by_gdp']).describe()

count    2.073600e+07
mean     2.027841e+00
std      1.958594e+00
min      2.308995e-05
25%      2.826754e-01
50%      1.328786e+00
75%      2.929156e+00
max      6.195986e+00
dtype: float64

## % increase taking maintenance into account

In [142]:
100*(reg_pct_alltype_all_modes['cost_diff']/reg_pct_alltype_all_modes['divided_by_gdp']).describe()

count    2.073600e+07
mean     7.069894e-01
std      6.828204e-01
min      3.893939e-07
25%      1.323723e-01
50%      4.688102e-01
75%      1.186896e+00
max      2.981399e+00
dtype: float64

In [143]:
reg_pct_alltype_all_modes.to_csv("reg_pct_alltype_all_modes.csv")

In [144]:
100*(reg_pct_alltype_all_modes['cost_diff']/reg_pct_alltype_all_modes['divided_by_gdp']).mean()

0.7069893844909984

In [145]:
100*(reg_pct_alltype_all_modes['cost_diff_all']/reg_pct_alltype_all_modes['divided_by_gdp']).mean()

8.853197511462769

In [146]:
100*(reg_pct_alltype_all_modes['cost_diff_all']/reg_pct_alltype_all_modes['divided_by_gdp']).describe()

count    2.073600e+07
mean     8.853198e+00
std      7.472475e+00
min      6.469521e-06
25%      2.032429e+00
50%      7.350708e+00
75%      1.532245e+01
max      2.902259e+01
dtype: float64

In [147]:
reg_pct_data_res_all_modes.loc[reg_pct_data_res_all_modes['cost_diff'].idxmax()]

costtype                          new
divided_by_gdp              0.0224678
cost_increase                    0.96
initial_risk              9.10765e-05
final_risk                3.14941e-05
share_exposed                0.649294
divided_by_gdp_res           0.023768
divided_by_gdp_res_all        0.03248
cost_diff                  0.00130018
cost_diff_all               0.0100122
Name: (164, 2, 3, 1, 1, 1, 2, 2, 1, 600, 5000, 35, 0, 1, ASIA), dtype: object

In [148]:
reg_pct_data_res_all_modes.loc[reg_pct_data_res_all_modes['cost_diff_all'].idxmax()]

costtype                          new
divided_by_gdp              0.0265286
cost_increase                    0.96
initial_risk              4.72235e-05
final_risk                1.57412e-05
share_exposed                0.300945
divided_by_gdp_res          0.0273157
divided_by_gdp_res_all      0.0396049
cost_diff                 0.000787051
cost_diff_all               0.0130763
Name: (98, 2, 1, 1, 1, 1, 1, 2, 0, 600, 5000, 35, 0, 1, CIS), dtype: object

In [149]:
reg_pct_data_res_all_modes.reset_index().columns

Index(['combi', 'ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'modal_scenario',
       'ur_ideal', 'urt_ideal', 'delay', 'costs_evolution_road',
       'costs_evolution_rail', 'region', 'costtype', 'divided_by_gdp',
       'cost_increase', 'initial_risk', 'final_risk', 'share_exposed',
       'divided_by_gdp_res', 'divided_by_gdp_res_all', 'cost_diff',
       'cost_diff_all'],
      dtype='object')

In [150]:
%reload_ext autoreload
from convert_pc_to_usd import load_GDP,return_usd

In [151]:
GDP_file = 'C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx'

In [152]:
GDP2 = load_GDP(GDP_file)

In [153]:
GDP2.sample(3)

,wbregionname,East Asia and Pacific,Eastern Europe and Central Asia,Latin America and Caribbean,Middle-East and North Africa,South Asia,Sub-Saharan Africa,Africa,Asia,all_countries
year,ssp,,,,,,,,,
2025,ssp2,2.421799e+13,4.702948e+12,1.125203e+13,3.381403e+12,6.666322e+12,4.172937e+12,7.554340e+12,3.088431e+13,5.439364e+13
2024,ssp5,2.550277e+13,4.759305e+12,1.135662e+13,3.390097e+12,6.789933e+12,4.253129e+12,7.643226e+12,3.229270e+13,5.605185e+13
2026,ssp1,2.763252e+13,5.026609e+12,1.195269e+13,3.583024e+12,7.477713e+12,4.680610e+12,8.263635e+12,3.511023e+13,6.035317e+13


In [154]:
reg_pct_data_res_all_modes = reg_pct_data_res_all_modes.reset_index()

reg_pct_data_res_all_modes["region"] = reg_pct_data_res_all_modes.region.replace({"MAF":"Africa",\
                                      "LAM":"Latin America and Caribbean",\
                                      "CIS":"Eastern Europe and Central Asia",\
                                      "ASIA":"Asia",\
                                      "OCDE":"OECD"})

In [155]:
reg_pct_alltype_all_modes = reg_pct_alltype_all_modes.reset_index()

reg_pct_alltype_all_modes["region"] = reg_pct_alltype_all_modes.region.replace({"MAF":"Africa",\
                                      "LAM":"Latin America and Caribbean",\
                                      "CIS":"Eastern Europe and Central Asia",\
                                      "ASIA":"Asia",\
                                      "OCDE":"OECD"})

In [156]:
test = return_usd(reg_pct_alltype_all_modes,'region','divided_by_gdp',indexes_inv,GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [157]:
test_tot = test.set_index(indexes_inv+['region']).sum(level=indexes_inv)

In [158]:
test_tot['cost_bUSD_disc_annual'].describe()

count    41472.000000
mean      1034.659138
std        211.699739
min        602.398846
25%        870.293502
50%       1003.814582
75%       1172.865212
max       1792.684520
Name: cost_bUSD_disc_annual, dtype: float64

In [159]:
reg_usd_data_res_all_modes = return_usd(reg_pct_data_res_all_modes,'region','cost_diff',indexes_inv,GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [160]:
reg_usd_data_res_all_modes.region.unique()

array(['Asia', 'Eastern Europe and Central Asia',
       'Latin America and Caribbean', 'Africa'], dtype=object)

In [161]:
tot_usd_data_res_all_modes = reg_usd_data_res_all_modes.set_index(indexes_inv+['region']).sum(level=indexes_inv)

## range of cost increase if exposed assets are upgraded

In [162]:
tot_usd_data_res_all_modes.loc[tot_usd_data_res_all_modes['cost_bUSD_disc_annual'].idxmin()]*1000

cost_bUSD_disc           12901.550857
cost_bUSD_disc_annual      860.103390
Name: (301, 4, 1, 1, 2, 2, 1, 1, 0, 900, 30000, 65, 2, 0), dtype: float64

In [163]:
tot_usd_data_res_all_modes.loc[tot_usd_data_res_all_modes['cost_bUSD_disc_annual'].idxmax()]

cost_bUSD_disc           526.117356
cost_bUSD_disc_annual     35.074490
Name: (164, 2, 3, 1, 1, 1, 2, 2, 1, 600, 5000, 35, 1, 1), dtype: float64

In [164]:
reg_usd_data_res_all_all_modes = return_usd(reg_pct_data_res_all_modes,'region','cost_diff_all',indexes_inv,GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [165]:
tot_usd_data_res_all_all_modes = reg_usd_data_res_all_all_modes.set_index(indexes_inv+['region']).sum(level=indexes_inv)

## range of cost increase if all assets are upgraded

In [166]:
tot_usd_data_res_all_all_modes.loc[tot_usd_data_res_all_all_modes['cost_bUSD_disc_annual'].idxmin()]

cost_bUSD_disc           128.881533
cost_bUSD_disc_annual      8.592102
Name: (301, 4, 1, 1, 2, 2, 1, 1, 0, 900, 30000, 65, 2, 0), dtype: float64

In [167]:
tot_usd_data_res_all_all_modes.loc[tot_usd_data_res_all_all_modes['cost_bUSD_disc_annual'].idxmax()]

cost_bUSD_disc           5257.537518
cost_bUSD_disc_annual     350.502501
Name: (164, 2, 3, 1, 1, 1, 2, 2, 1, 600, 5000, 35, 1, 1), dtype: float64

In [168]:
best_scenario = tot_usd_data_res_all_modes.reset_index()
select = (best_scenario.urt_ideal==30000)&(best_scenario.modal_scenario==1)&(best_scenario.ind_climat==2)&(best_scenario.ind_mitig==2)&(best_scenario.ur_ideal==600)&\
(best_scenario.ind_trans_A==2)&(best_scenario.ind_trans_S==2)&(best_scenario.ind_trans_I==2)&(best_scenario.ind_trans_F==2)&(best_scenario.delay==35)&\
(best_scenario.costs_evolution_road==2)&(best_scenario.costs_evolution_rail==2)&(best_scenario.ind_growth==2)

best_scenario_exposed = best_scenario.loc[select,:]
best_scenario_exposed['scenario']='best'

C:\ProgramData\Anaconda3\envs\Julie1\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [169]:
extract = best_scenario_exposed
extract = extract.append(tot_usd_data_res_all_modes.reset_index().loc[tot_usd_data_res_all_modes.reset_index()['cost_bUSD_disc_annual'].idxmax()])
extract = extract.append(tot_usd_data_res_all_modes.reset_index().loc[tot_usd_data_res_all_modes.reset_index()['cost_bUSD_disc_annual'].idxmin()])

extract['exposed'] = 'yes'

In [170]:
extract

,combi,ind_climat,ind_growth,ind_mitig,ind_trans_A,ind_trans_S,ind_trans_I,ind_trans_F,modal_scenario,ur_ideal,urt_ideal,delay,costs_evolution_road,costs_evolution_rail,cost_bUSD_disc,cost_bUSD_disc_annual,scenario,exposed
23034,160.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,600.0,30000.0,35.0,2.0,2.0,127.462742,8.497516,best,yes
23576,164.0,2.0,3.0,1.0,1.0,1.0,2.0,2.0,1.0,600.0,5000.0,35.0,1.0,1.0,526.117356,35.074490,NaN,yes
29399,301.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,900.0,30000.0,65.0,2.0,0.0,12.901551,0.860103,NaN,yes


In [171]:
best_scenario = tot_usd_data_res_all_all_modes.reset_index()
select = (best_scenario.urt_ideal==30000)&(best_scenario.modal_scenario==1)&(best_scenario.ind_climat==2)&(best_scenario.ind_mitig==2)&(best_scenario.ur_ideal==600)&\
(best_scenario.ind_trans_A==2)&(best_scenario.ind_trans_S==2)&(best_scenario.ind_trans_I==2)&(best_scenario.ind_trans_F==2)&(best_scenario.delay==35)&\
(best_scenario.costs_evolution_road==2)&(best_scenario.costs_evolution_rail==2)&(best_scenario.ind_growth==2)

best_scenario_all = best_scenario.loc[select,:]
best_scenario_exposed['scenario']='best'

In [172]:
extract2 = best_scenario_all
extract2 = extract2.append(tot_usd_data_res_all_all_modes.reset_index().loc[tot_usd_data_res_all_all_modes.reset_index()['cost_bUSD_disc_annual'].idxmax()])
extract2 = extract2.append(tot_usd_data_res_all_all_modes.reset_index().loc[tot_usd_data_res_all_all_modes.reset_index()['cost_bUSD_disc_annual'].idxmin()])

extract2['exposed'] = 'no'

In [173]:
extract2.append(extract)

C:\ProgramData\Anaconda3\envs\Julie1\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,combi,cost_bUSD_disc,cost_bUSD_disc_annual,costs_evolution_rail,costs_evolution_road,delay,exposed,ind_climat,ind_growth,ind_mitig,ind_trans_A,ind_trans_F,ind_trans_I,ind_trans_S,modal_scenario,scenario,ur_ideal,urt_ideal
23034,160.0,1076.172250,71.744817,2.0,2.0,35.0,no,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,600.0,30000.0
23576,164.0,5257.537518,350.502501,1.0,1.0,35.0,no,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,NaN,600.0,5000.0
29399,301.0,128.881533,8.592102,0.0,2.0,65.0,no,4.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,NaN,900.0,30000.0
23034,160.0,127.462742,8.497516,2.0,2.0,35.0,yes,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,best,600.0,30000.0
23576,164.0,526.117356,35.074490,1.0,1.0,35.0,yes,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,NaN,600.0,5000.0
29399,301.0,12.901551,0.860103,0.0,2.0,65.0,yes,4.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,NaN,900.0,30000.0


In [174]:
reg_usd_risk_ini = return_usd(reg_pct_data_res_all_modes,'region','initial_risk',indexes_inv,GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [175]:
reg_usd_risk_final = return_usd(reg_pct_data_res_all_modes,'region','final_risk',indexes_inv,GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [176]:
reg_usd_risk_ini.sample(3)

,combi,ind_climat,ind_growth,ind_mitig,ind_trans_A,ind_trans_S,ind_trans_I,ind_trans_F,modal_scenario,ur_ideal,urt_ideal,delay,costs_evolution_road,costs_evolution_rail,region,cost_bUSD_disc,cost_bUSD_disc_annual
163648,369,4,3,2,1,1,1,1,0,600,5000,35,2,2,Africa,13.952379,0.930159
138060,95,0,3,2,2,2,2,1,1,900,5000,35,1,1,Africa,10.509555,0.700637
145564,147,2,2,2,1,1,2,1,1,900,5000,35,0,2,Africa,11.543805,0.769587


In [177]:
tot_usd_risk_final = reg_usd_risk_final.set_index(indexes_inv+['region']).sum(level=indexes_inv)
tot_usd_risk_ini = reg_usd_risk_ini.set_index(indexes_inv+['region']).sum(level=indexes_inv)

In [178]:
risk_reduction = tot_usd_risk_ini/tot_usd_risk_final

In [179]:
risk_reduction.cost_bUSD_disc.describe()

count    41472.000000
mean         2.242666
std          0.121913
min          2.015298
25%          2.141749
50%          2.235175
75%          2.338634
max          2.555910
Name: cost_bUSD_disc, dtype: float64

In [180]:
extract3 = extract2.append(extract)

C:\ProgramData\Anaconda3\envs\Julie1\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [181]:
extract3 = extract3.merge(risk_reduction.cost_bUSD_disc.reset_index().rename(columns={"cost_bUSD_disc":"risk_reduction"}),on=indexes_inv)

In [182]:
extract3.to_csv("three_resilience_scenarios_transport.csv")

In [183]:
extract3

,combi,cost_bUSD_disc,cost_bUSD_disc_annual,costs_evolution_rail,costs_evolution_road,delay,exposed,ind_climat,ind_growth,ind_mitig,ind_trans_A,ind_trans_F,ind_trans_I,ind_trans_S,modal_scenario,scenario,ur_ideal,urt_ideal,risk_reduction
0,160.0,1076.172250,71.744817,2.0,2.0,35.0,no,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,600.0,30000.0,2.185783
1,160.0,127.462742,8.497516,2.0,2.0,35.0,yes,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,best,600.0,30000.0,2.185783
2,164.0,5257.537518,350.502501,1.0,1.0,35.0,no,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,NaN,600.0,5000.0,2.396657
3,164.0,526.117356,35.074490,1.0,1.0,35.0,yes,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,NaN,600.0,5000.0,2.396657
4,301.0,128.881533,8.592102,0.0,2.0,65.0,no,4.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,NaN,900.0,30000.0,2.037521
5,301.0,12.901551,0.860103,0.0,2.0,65.0,yes,4.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,NaN,900.0,30000.0,2.037521
